In [12]:
!pip install simclr

In [13]:
import torch
import torchvision

import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms

from simclr.modules.transformations import TransformsSimCLR
from simclr.modules import NT_Xent
from simclr.modules.identity import Identity

from torch.utils.data.dataloader import DataLoader

import gc
gc.collect()
torch.cuda.empty_cache()

In [14]:
class SimCLR(nn.Module):

    def __init__(self, encoder, projection_dim, n_features):
        super(SimCLR, self).__init__()

        self.encoder = encoder
        self.n_features = n_features

        self.encoder.fc = Identity()
        
        self.projector = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False, device = DEVICE),
            nn.ReLU(),
            nn.Linear(self.n_features, self.n_features, bias=False, device = DEVICE),
            nn.ReLU(),
        )

    def forward(self, x_i, x_j):
        
        
        h_i = self.encoder(x_i)
        h_j = self.encoder(x_j)
        
        z_i = self.projector(h_i)
        z_j = self.projector(h_j)
        
        x_i = self.encoder.conv1(x_i)
        x_i = self.encoder.bn1(x_i)
        x_i = self.encoder.relu(x_i)
        x_i = self.encoder.maxpool(x_i)
        
        
        #Extract the features from intermediate layers
        h_1 = self.encoder.layer1(x_i)
        h_2 = self.encoder.layer2(h_1)
        h_3 = self.encoder.layer3(h_2)
        h_4 = self.encoder.layer4(h_3)
        
        h_1.norm(dim = 1, p = 2)
        h_2.norm(dim = 1, p = 2)
        h_3.norm(dim = 1, p = 2)
        h_4.norm(dim = 1, p = 2)
        
        return h_i, h_j, z_i, z_j, h_1, h_2, h_3, h_4

In [15]:
image_size = 14
batch_size = 128
projection_dim = 64 #128 for imagenette, 64 for CIFAR10

In [16]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
tim_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

train_transform = TransformsSimCLR(size=image_size)
test_transform = tim_transform

train_dataset = torchvision.datasets.CIFAR10(root = '/content/sample_data', transform = train_transform, download = True)
test_dataset = torchvision.datasets.CIFAR10(root = '/content/sample_data', transform = test_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last=True,
    num_workers = 2,
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle = False,
    drop_last=True,
    num_workers = 2,
)

Files already downloaded and verified


In [18]:
encoder = torchvision.models.resnet18()
n_features = encoder.fc.in_features  # get dimensions of last fully-connected layer
model = SimCLR(encoder, projection_dim, n_features).to(DEVICE)

In [19]:
import os

from simclr.modules import LARS


def load_optimizer(optimizer, model, batch_size, weight_decay, epochs):
    if optimizer == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)  # TODO: LARS
    elif optimizer == "LARS":
        learning_rate = 0.3 * batch_size / 256
        optimizer = LARS(
            model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
            exclude_from_weight_decay=["batch_normalization", "bias"],
        )

        # "decay the learning rate with the cosine decay schedule without restarts"
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, epochs, eta_min=0, last_epoch=-1
        )
    else:
        raise NotImplementedError

    return optimizer, scheduler


def save_model(current_epoch, model, optimizer, model_path):
    out = os.path.join(model_path, "checkpoint_{}.tar".format(current_epoch))

    if isinstance(model, torch.nn.DataParallel):
        torch.save(model.module.state_dict(), out)
    else:
        torch.save(model.state_dict(), out)


In [20]:
from simclr.modules import NT_Xent
from tqdm import tqdm

weight_decay = 1e-6
epochs = 100
start_epoch = 0
model_path = '/kaggle/working'

optimizer, scheduler = load_optimizer('LARS', model, batch_size, weight_decay, epochs)
criterion = NT_Xent(batch_size, temperature = 0.5, world_size = 1)

In [21]:
def train(train_loader, model, criterion, optimizer, global_step):
    loss_epoch = 0
    for step, ((x_i, x_j), _) in enumerate(train_loader):
        optimizer.zero_grad()
        x_i = x_i.cuda()
        x_j = x_j.cuda()

        # positive pair, with encoding
        h_i, h_j, z_i, z_j, _, _, _, _ = model(x_i, x_j)
        
        loss = criterion(z_i, z_j)
        loss.backward()

        optimizer.step()

        if step % 50 == 0:
            print(f"Step [{step}/{len(train_loader)}]\t Loss: {loss.item()}")

        loss_epoch += loss.item()
        global_step += 1
    return loss_epoch

In [22]:
global_step = 0
current_epoch = 0
for epoch in tqdm(range(start_epoch, epochs)):
    lr = optimizer.param_groups[0]["lr"]
    loss_epoch = train(train_loader, model, criterion, optimizer, global_step)

    if scheduler:
        scheduler.step()

    # save every 10 epochs
    if epoch % 10 == 0:
        save_model(current_epoch, model, optimizer, model_path)

    print(
        f"Epoch [{epoch}/{epochs}]\t Loss: {loss_epoch / len(train_loader)}\t lr: {round(lr, 5)}"
    )
    current_epoch += 1

# end training
save_model(current_epoch, model, optimizer, model_path)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/simclr/modules/lars.py:137: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/python_arg_parser.cpp:1055.)
  next_v.mul_(momentum).add_(scaled_lr, grad)


Step [0/390]	 Loss: 5.530717372894287
Step [50/390]	 Loss: 5.477902412414551
Step [100/390]	 Loss: 5.276640892028809
Step [150/390]	 Loss: 5.25023889541626
Step [200/390]	 Loss: 5.195037364959717
Step [250/390]	 Loss: 5.123541831970215
Step [300/390]	 Loss: 5.110691070556641
Step [350/390]	 Loss: 5.073017597198486


  1%|          | 1/100 [01:19<2:10:27, 79.07s/it]

Epoch [0/100]	 Loss: 5.261579210330279	 lr: 0.15
Step [0/390]	 Loss: 5.2655792236328125
Step [50/390]	 Loss: 5.166264533996582
Step [100/390]	 Loss: 5.018362998962402
Step [150/390]	 Loss: 5.161472320556641
Step [200/390]	 Loss: 5.114624500274658
Step [250/390]	 Loss: 5.095117092132568
Step [300/390]	 Loss: 5.049841403961182
Step [350/390]	 Loss: 5.027430534362793


  2%|▏         | 2/100 [02:38<2:09:31, 79.30s/it]

...

Epoch [97/100]	 Loss: 4.76603589791518	 lr: 0.00033
Step [0/390]	 Loss: 4.845604419708252
Step [50/390]	 Loss: 4.773802757263184
Step [100/390]	 Loss: 4.795609951019287
Step [150/390]	 Loss: 4.7747931480407715
Step [200/390]	 Loss: 4.7818145751953125
Step [250/390]	 Loss: 4.7421746253967285
Step [300/390]	 Loss: 4.835600852966309
Step [350/390]	 Loss: 4.788452625274658


 99%|█████████▉| 99/100 [2:10:01<01:17, 77.82s/it]

Epoch [98/100]	 Loss: 4.769894914138011	 lr: 0.00015
Step [0/390]	 Loss: 4.8128790855407715
Step [50/390]	 Loss: 4.794370174407959
Step [100/390]	 Loss: 4.7896552085876465
Step [150/390]	 Loss: 4.770269870758057
Step [200/390]	 Loss: 4.731128692626953
Step [250/390]	 Loss: 4.7817206382751465
Step [300/390]	 Loss: 4.851526260375977
Step [350/390]	 Loss: 4.667038917541504


100%|██████████| 100/100 [2:11:19<00:00, 78.79s/it]

Epoch [99/100]	 Loss: 4.764617267021766	 lr: 4e-05


In [23]:
from simclr.modules import LogisticRegression

def train(loader, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()

        x = x.to(DEVICE)
        y = y.to(DEVICE)

        output = model(x)
        loss = criterion(output, y)

        predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()

    return loss_epoch, accuracy_epoch

In [24]:
def test(loader, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model.eval()
    for step, (x, y) in enumerate(loader):
        model.zero_grad()

        x = x.to(DEVICE)
        y = y.to(DEVICE)

        output = model(x)
        loss = criterion(output, y)

        predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss_epoch += loss.item()

    return loss_epoch, accuracy_epoch

In [25]:
epoch_num = 100
logistic_epochs = 800

In [26]:
train_transform = tim_transform
test_transform = train_transform

train_dataset = torchvision.datasets.CIFAR10(root = '/content/sample_data', transform = train_transform, download = True)
test_dataset = torchvision.datasets.CIFAR10(root = '/content/sample_data', transform = test_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last=True,
    num_workers = 2,
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle = False,
    drop_last=True,
    num_workers = 2,
)

Files already downloaded and verified


In [27]:
encoder = torchvision.models.resnet18()

n_features = encoder.fc.in_features
simclr_model = SimCLR(encoder, projection_dim, n_features)
model_fp = os.path.join(
    model_path, "checkpoint_{}.tar".format(epochs)
)
simclr_model.load_state_dict(torch.load(model_fp, map_location=DEVICE.type))
simclr_model.to(DEVICE)

SimCLR(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [28]:
## Logistic Regression
n_classes = 10 
model = LogisticRegression(simclr_model.n_features, n_classes)
model = model.to(DEVICE)

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()

In [30]:
def inference(loader, simclr_model, device):
    feature_vector = []
    labels_vector = []
    for step, (x, y) in enumerate(loader):
        x = x.to(device)

        # get encoding
        with torch.no_grad():
            h, _, z, _, _, _, _, _ = simclr_model(x, x)

        h = h.detach()

        feature_vector.extend(h.cpu().detach().numpy())
        labels_vector.extend(y.numpy())

        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")

    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector


def get_features(context_model, train_loader, test_loader, device):
    train_X, train_y = inference(train_loader, context_model, device)
    test_X, test_y = inference(test_loader, context_model, device)
    return train_X, train_y, test_X, test_y


def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test, batch_size):
    train = torch.utils.data.TensorDataset(
        torch.from_numpy(X_train), torch.from_numpy(y_train)
    )
    train_loader = torch.utils.data.DataLoader(
        train, batch_size=batch_size, shuffle=False
    )

    test = torch.utils.data.TensorDataset(
        torch.from_numpy(X_test), torch.from_numpy(y_test)
    )
    test_loader = torch.utils.data.DataLoader(
        test, batch_size=batch_size, shuffle=False
    )
    return train_loader, test_loader

In [31]:
print("### Creating features from pre-trained context model ###")
(train_X, train_y, test_X, test_y) = get_features(
    simclr_model, train_loader, test_loader, DEVICE
)

arr_train_loader, arr_test_loader = create_data_loaders_from_arrays(
    train_X, train_y, test_X, test_y, batch_size
)

### Creating features from pre-trained context model ###
Step [0/390]	 Computing features...
Step [20/390]	 Computing features...
Step [40/390]	 Computing features...
Step [60/390]	 Computing features...
Step [80/390]	 Computing features...
Step [100/390]	 Computing features...
Step [120/390]	 Computing features...
Step [140/390]	 Computing features...
Step [160/390]	 Computing features...
Step [180/390]	 Computing features...
Step [200/390]	 Computing features...
Step [220/390]	 Computing features...
Step [240/390]	 Computing features...
Step [260/390]	 Computing features...
Step [280/390]	 Computing features...
Step [300/390]	 Computing features...
Step [320/390]	 Computing features...
Step [340/390]	 Computing features...
Step [360/390]	 Computing features...
Step [380/390]	 Computing features...
Features shape (49920, 512)
Step [0/390]	 Computing features...
Step [20/390]	 Computing features...
Step [40/390]	 Computing features...
Step [60/390]	 Computing features...
Step [80/390]	

In [32]:
for epoch in tqdm(range(logistic_epochs)):
    loss_epoch, accuracy_epoch = train(arr_train_loader, model, criterion, optimizer)
    
    if epoch % 10 == 0:
      print(f"Epoch [{epoch}/{logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")


# final testing
loss_epoch, accuracy_epoch = test(
    arr_test_loader, model, criterion, optimizer
)
print(
    f"[FINAL]\t Loss: {loss_epoch / len(test_loader)}\t Accuracy: {accuracy_epoch / len(test_loader)}"
)

  0%|          | 1/800 [00:00<09:41,  1.37it/s]

Epoch [0/800]	 Loss: 1.6693740520721827	 Accuracy: 0.39503205128205127


  1%|▏         | 11/800 [00:08<09:46,  1.35it/s]

Epoch [10/800]	 Loss: 1.4587524163417327	 Accuracy: 0.47584134615384616


  3%|▎         | 21/800 [00:15<09:09,  1.42it/s]

...

Epoch [780/800]	 Loss: 1.3684184493162692	 Accuracy: 0.5090344551282051


 99%|█████████▉| 791/800 [09:27<00:06,  1.44it/s]

Epoch [790/800]	 Loss: 1.368415880508912	 Accuracy: 0.5090544871794872


100%|██████████| 800/800 [09:34<00:00,  1.39it/s]


[FINAL]	 Loss: 1.3613630022758092	 Accuracy: 0.5117387820512821
